In [1]:
import requests
from bs4 import BeautifulSoup
import time
import csv

def fetch_restaurant_info(num_links):
    URL = "https://www.tripadvisor.fr/Restaurants-g187147-Paris_Ile_de_France.html"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    response = requests.get(URL, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Trouver les liens de restaurants 
    restaurant_link_elements = soup.select('a[target="_blank"]')
    # Supprimer les éléments comme #REVIEWS des liens
    cleaned_links = ['https://www.tripadvisor.fr' + elem['href'].split('#')[0] for elem in restaurant_link_elements if 'Restaurant_Review' in elem['href']]
    
    print(len(cleaned_links))
    # Utilisez un ensemble pour éliminer les doublons
    restaurant_links = list(set(cleaned_links))
    
    # Boucle liens
    restaurant_links = restaurant_links[:num_links]
    restaurant_data = []

    for link in restaurant_links:
        response = requests.get(link, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Récupérer le nom du restaurant
        restaurant_name_elem = soup.select_one('h1[data-test-target="top-info-header"]')
        if restaurant_name_elem:
            restaurant_name = restaurant_name_elem.text.strip()

        # Récupérer notes
        restaurant_notes_elem = soup.select_one('.ZDEqb')
        if restaurant_notes_elem:
            restaurant_notes = restaurant_notes_elem.text.strip()
        else:
            restaurant_notes = None

        # Récupérer l'adresse du restaurant
        restaurant_address_elem = soup.select_one('.yEWoV')
        if restaurant_address_elem:
            restaurant_address = restaurant_address_elem.text.strip()
        else:
            restaurant_address = None
        
        # Récupérer prix
        restaurant_prix_elem = soup.select_one('.SrqKb')
        if restaurant_prix_elem:
            restaurant_prix = restaurant_prix_elem.text.strip()
        else:
            restaurant_prix = None


        restaurant_data.append({
            'NOM': restaurant_name,
            'ADRESSE': restaurant_address,
            'NOTE': restaurant_notes,
            'FOURCHETTE DE PRIX': restaurant_prix,
        })

        time.sleep(5)

    return restaurant_data


restaurants = fetch_restaurant_info(1000)



def save_to_csv(data, filename):
    keys = data[0].keys()
    with open(filename, 'w', newline='', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(data)

save_to_csv(restaurants, 'restaurants.csv')




421
